In [5]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product

### Problem Data

In [29]:
supply = [10000, 15000, 15000]
demand = [8000, 8000, 12000, 10000]
distributors = [0,0]
# check supply and demand balance (surplus exists)
net = sum(supply)-sum(demand)
num_supply = len(supply)
num_demand = len(demand)
num_distributor = len(distributors)
cartesina_supply_dist = list(product(range(num_supply), range(num_distributor)))
cartesina_dist_demand = list(product(range(num_distributor), range(num_demand)))
transportation_cost_supply_dist = [0.4, 0.32, 0.22, 0.24, 0.52, 0.36]
transportation_cost_dist_demand = [0.18, 0.34, 0.16, 0.36, 0.24, 0.48, 0.14, 0.3]
arc_cost_supply_dist = {cartesina_supply_dist[i]:transportation_cost_supply_dist[i] for i in range(6)}
arc_cost_dist_demand = {cartesina_dist_demand[i]:transportation_cost_dist_demand[i] for i in range(8)}
print(arc_cost_supply_dist)
print(arc_cost_dist_demand)

{(0, 0): 0.4, (0, 1): 0.32, (1, 0): 0.22, (1, 1): 0.24, (2, 0): 0.52, (2, 1): 0.36}
{(0, 0): 0.18, (0, 1): 0.34, (0, 2): 0.16, (0, 3): 0.36, (1, 0): 0.24, (1, 1): 0.48, (1, 2): 0.14, (1, 3): 0.3}


### Decision Variable

In [30]:
model = gp.Model('Trasshipment Network')

In [32]:
num_product_supply_to_dist = model.addVars(cartesina_supply_dist, vtype=GRB.CONTINUOUS, lb=0.00, name='#supply_to_dist')
num_product_dist_to_demand = model.addVars(cartesina_dist_demand, vtype=GRB.CONTINUOUS, lb=0.00, name='#dist_to_demand')
print(num_product_supply_to_dist)
print(num_product_dist_to_demand)

{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model Update*>, (1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (2, 0): <gurobi.Var *Awaiting Model Update*>, (2, 1): <gurobi.Var *Awaiting Model Update*>}
{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model Update*>, (0, 2): <gurobi.Var *Awaiting Model Update*>, (0, 3): <gurobi.Var *Awaiting Model Update*>, (1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>}


### Objective Function

In [33]:
model.setObjective(gp.quicksum(num_product_supply_to_dist[(i,k)]*arc_cost_supply_dist[(i,k)]
                               for i in range(num_supply) for k in range(num_distributor))+
                  gp.quicksum(num_product_dist_to_demand[(k,j)]*arc_cost_dist_demand[(k,j)]
                              for k in range(num_distributor) for j in range(num_demand)
                             ))

### Constraint

In [35]:
# supply
supply_constr = model.addConstrs((gp.quicksum(num_product_supply_to_dist[(i,k)] for k in range(num_distributor))<=supply[i]
                                 for i in range(num_supply)), name='Supply')
# demand
demand_constr = model.addConstrs((gp.quicksum(num_product_dist_to_demand[(k,j)] for k in range(num_distributor)) >= demand[j]
                                 for j in range(num_demand)), name='Demand')
# conservation law
conservation_constr = model.addConstrs((gp.quicksum(num_product_supply_to_dist[(i,k)] for i in range(num_supply))==
                                       gp.quicksum(num_product_dist_to_demand[(k,j)] for j in range(num_demand))
                                       for k in range(num_distributor)), name='Conservation Law')            

In [36]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 16 rows, 28 columns and 42 nonzeros
Model fingerprint: 0x72b41f5f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+03, 2e+04]
Presolve removed 7 rows and 14 columns
Presolve time: 0.02s
Presolved: 9 rows, 14 columns, 28 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.750000e+03   0.000000e+00      0s
      10    2.0080000e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.04 seconds (0.00 work units)
Optimal objective  2.008000000e+04


In [37]:
model.printAttr('X')


    Variable            X 
-------------------------
#supply_to_dist[0,1]        10000 
#supply_to_dist[1,0]        15000 
#supply_to_dist[2,1]        13000 
#dist_to_demand[0,0]         7000 
#dist_to_demand[0,1]         8000 
#dist_to_demand[1,0]         1000 
#dist_to_demand[1,2]        12000 
#dist_to_demand[1,3]        10000 
